In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from mlxtend.evaluate import (
    cochrans_q,
    mcnemar,
    mcnemar_table
)
from itertools import combinations

Prepare data for hypothesis testing.

In [8]:
df_approach_1 = pd.read_csv("../data/classification_results_approach_1.csv")
df_approach_2 = pd.read_csv("../data/classification_results_approach_2.csv")
df_approach_3 = pd.read_csv("../data/classification_results_approach_3.csv")

In [ ]:
def get_label_combinations(df):
    return list(combinations(df.drop(columns='truth'), 2))

def get_value_combinations(listOfValues):
    return list(combinations(listOfValues, 2))

In [ ]:
def mcnemar_test(df: pd.DataFrame, values, labels):
    print(f"Comparison of {labels[0]} and {labels[1]}\n")

    tb = mcnemar_table(
        np.array(df['truth']),
        np.array(values[0]),
        np.array(values[1])
    )

    print(f"Confusion matrix:\n{tb}\n")

    chi2, p = mcnemar(tb, corrected=True)

    print(f"chi-squared: {chi2:.2f} \np-value: {p}")
    print("----------")

### Approach 1

In [9]:
df_approach_1

,truth,svc_linear,svc_poly,poly_reg
0,30,1,1,62.485990
1,19,13,3,56.882629
2,115,13,13,60.267201
3,41,1,1,62.866345
4,3,13,13,62.268922
...,...,...,...,...
305,3,1,1,60.446078
306,23,1,1,58.477346
307,132,1,1,62.715579
308,1,1,1,61.271565


In [57]:
col_names_1 = df_approach_1.columns

SyntaxError: invalid decimal literal (403049408.py, line 1)

In [56]:
# check cochrans q for multiple classifier
q, p = cochrans_q(
        np.array(df_approach_1[col_names_1[0]]),
        np.array(df_approach_1[col_names_1[1]]),
        np.array(df_approach_1[col_names_1[2]]),
        np.array(df_approach_1[col_names_1[3]]),
)

print(f"Q (or chi-squared): {q:.2f} \np-value: {p}")

Q (or chi-squared): 12.25 
p-value: 0.002187491118182885


In [51]:
value_list = list()
value_list.append(list(df_approach_1[col_names_1[1]]))
value_list.append(list(df_approach_1[col_names_1[2]]))
value_list.append(list(df_approach_1[col_names_1[3]]))

labels = get_label_combinations(df_approach_1)
values = get_value_combinations(value_list)

In [52]:
for i in range(len(labels)):
    mcnemar_test(df_approach_1, values[i], labels[i])

Comparison of svc_linear and svc_poly

Confusion matrix:
[[  6   1]
 [  1 302]]

chi-squared: 0.50 
p-value: 0.47950012218695337
----------
Comparison of svc_linear and poly_reg

Confusion matrix:
[[  0   7]
 [  0 303]]

chi-squared: 5.14 
p-value: 0.02334220201289086
----------
Comparison of svc_poly and poly_reg

Confusion matrix:
[[  0   7]
 [  0 303]]

chi-squared: 5.14 
p-value: 0.02334220201289086
----------


### Approach 2

In [58]:
df_approach_2

,truth,SVC_linear,SVC_rbf
0,54,8,8
1,15,10,1
2,43,9,9
3,20,54,5
4,11,13,10
...,...,...,...
305,5,17,17
306,1,20,20
307,136,85,9
308,14,23,23


In [60]:
col_names_2 = df_approach_2.columns
col_names_2

Index(['truth', 'SVC_linear', 'SVC_rbf'], dtype='object')

In [61]:
value_list = list()
value_list.append(df_approach_2[col_names_2[1]])
value_list.append(df_approach_2[col_names_2[2]])

In [63]:
labels = get_label_combinations(df_approach_2)
values = get_value_combinations(value_list)

for i in range(len(labels)):
    mcnemar_test(df_approach_2, values[i], labels[i])

Comparison of SVC_linear and SVC_rbf

Confusion matrix:
[[  2   2]
 [  4 302]]

chi-squared: 0.17 
p-value: 0.6830913983096086
----------


### Approach 3

In [65]:
df_approach_3

,truth,svm_linear,svm_poly,logistic_regression,k_nearest_neighbour
0,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_UP
1,NEW_ENTRY,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_DOWN
2,MOVE_DOWN,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_DOWN
3,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_UP
4,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_UP
...,...,...,...,...,...
305,MOVE_DOWN,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_UP
306,MOVE_DOWN,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_UP
307,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_UP
308,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_UP,MOVE_DOWN


In [64]:
col_names_3 = df_approach_3.columns
col_names_3

Index(['truth', 'svm_linear', 'svm_poly', 'logistic_regression',
       'k_nearest_neighbour'],
      dtype='object')

In [ ]:
# cochrans q


In [ ]:
# prepare list of values

In [ ]:
# mcnemar 

# Hypothesis Testing
TODO: extend for every approach

$H_{0}$: There is no difference between the classification accuracy.

$H_{A}$: There are differences between the classification accuracy.

Significance threshold: $\alpha = 0.05$

This threshold is used to determine if the null hypothesis holds or can be rejected when compared to the p-value:
* $p > \alpha$ &rarr; $H_{0}$ is true
* $p \le \alpha$ &rarr; $H_{0}$ is false and $H_{A}$ is true